# Gestión de Ventana de Contexto - Límites de Memoria de la Bitácora

## 🌌 Escenario: USS Enterprise - Sistema de Bitácora Estelar

Los sistemas de bitácora de la USS Enterprise tienen **límites de memoria**. Cuando las entradas acumuladas exceden la capacidad, debemos decidir qué información conservar y cuál archivar.

En esta tarea aprenderás:
- 🔄 **Ventana Deslizante**: Mantener las últimas N entradas
- 💾 **Presupuesto de Tokens**: Optimizar según el tamaño real de la información
- 📉 **Truncamiento de Historial**: Reproducir contexto reciente en nuevos registros
- ⚖️ **Comparación de Estrategias**: Elegir el enfoque adecuado

---

## 🚀 Conceptos Clave

**Ventana de Contexto**: La cantidad máxima de información que un sistema puede mantener en memoria activa.

**Estrategias de Gestión**:
1. **Ventana Deslizante**: Mantener un número fijo de entradas recientes
2. **Basado en Tokens**: Mantener entradas hasta un límite de tamaño
3. **Truncamiento y Reproducción**: Crear nuevos registros con contexto esencial

🖖 *"La eficiencia lógica requiere optimización de recursos." - Spock*

## 📦 Paso 1: Importar Librerías

Importamos las herramientas necesarias para gestionar nuestro sistema de bitácora.

In [1]:
import asyncio
import os
from agent_framework.openai import OpenAIChatClient

print("✅ Librerías cargadas correctamente")

✅ Librerías cargadas correctamente


## ⚙️ Paso 2: Configurar el Cliente Azure OpenAI

Establecemos conexión con los sistemas centrales de la Federación.

In [2]:
# Configuración del cliente
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
model_id = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("🔧 Configuración del Sistema:")
print(f"   URL Base: {base_url}")
print(f"   ID del Modelo: {model_id}")

# Crear cliente
client = OpenAIChatClient(
    base_url=base_url,
    api_key=api_key,
    model_id=model_id
)

print("\n✅ Cliente conectado a los sistemas de la Federación")

🔧 Configuración del Sistema:
   URL Base: https://rnava-m697t629-eastus2.openai.azure.com/openai/v1/
   ID del Modelo: gpt-4

✅ Cliente conectado a los sistemas de la Federación


## 🤖 Paso 3: Crear el Sistema de Bitácora Estelar

Activamos el sistema de bitácora de la USS Enterprise con instrucciones para mantener respuestas concisas y optimizar la memoria.

In [3]:
# Crear sistema de bitácora
agent = client.create_agent(
    name="SistemaBitacoraEstelar",
    instructions="""Eres el sistema de bitácora de la USS Enterprise. Registras y recuperas información de forma concisa y 
    eficiente. Mantén las respuestas breves para optimizar la memoria."""
)

print("✅ SistemaBitacoraEstelar activado y listo para registrar")

✅ SistemaBitacoraEstelar activado y listo para registrar


## 🚀 Prueba 1: Simular Sesión Extendida de Bitácora

Vamos a simular una larga sesión de registro con 15 entradas secuenciales.

In [4]:
async def simular_sesion_larga():
    """Simula 15 entradas consecutivas en el sistema de bitácora"""
    print("🚀 Iniciando simulación de sesión extendida...\n")
    
    # Crear nuevo registro de bitácora
    bitacora = agent.get_new_thread()
    
    # Simular 15 entradas
    for i in range(1, 16):
        mensaje = f"Entrada de bitácora estelar número {i}. Sistema, confirma recepción."
        result = await agent.run(mensaje, thread=bitacora)
        print(f"   Turno {i}: ✓ Enviado y confirmado")
    
    print(f"\n✅ Sesión de bitácora completada. 15 entradas registradas.")
    return bitacora

# Ejecutar simulación
bitacora = await simular_sesion_larga()

🚀 Iniciando simulación de sesión extendida...

   Turno 1: ✓ Enviado y confirmado
   Turno 2: ✓ Enviado y confirmado
   Turno 3: ✓ Enviado y confirmado
   Turno 4: ✓ Enviado y confirmado
   Turno 5: ✓ Enviado y confirmado
   Turno 6: ✓ Enviado y confirmado
   Turno 7: ✓ Enviado y confirmado
   Turno 8: ✓ Enviado y confirmado
   Turno 9: ✓ Enviado y confirmado
   Turno 10: ✓ Enviado y confirmado
   Turno 11: ✓ Enviado y confirmado
   Turno 12: ✓ Enviado y confirmado
   Turno 13: ✓ Enviado y confirmado
   Turno 14: ✓ Enviado y confirmado
   Turno 15: ✓ Enviado y confirmado

✅ Sesión de bitácora completada. 15 entradas registradas.


In [ ]:
user_msg="Cual es la menor y mayor entrada registrada"
res = await agent.run(user_msg, thread=bitacora)
print(res.text)

La menor entrada registrada es la número 1.  
La mayor entrada registrada es la número 15.


# 🔄 Prueba 2: Reducción Automática de Historial (Ventana Deslizante Real)

## 📖 Concepto

Un **reductor automático** monitorea el `message_store` del thread REAL y aplica ventana deslizante.

**Cómo funciona:**
1. Se establece un **umbral** (ej: 10 mensajes)
2. Se define una **ventana** (ej: mantener últimos 5)
3. Cuando se supera el umbral, se extrae del thread REAL usando `message_store.list_messages()`
4. Se crean nuevos threads con solo los últimos N mensajes reproducidos
5. El agente continúa con contexto limitado pero suficiente

**Diferencia clave con Task 6:**
- Task 6: Extrae para análisis puntual
- Task 7: Crea ventana deslizante continua durante la conversación

**Caso Star Trek:** La bitácora de la USS Enterprise solo mantiene las últimas 5 entradas activas para optimizar memoria, pero accede al `message_store` REAL del thread para hacer la reducción.


In [5]:
class ReductorAutomaticoConMessageStore:
    """
    Reductor automático que usa message_store.list_messages() del thread REAL
    
    Esta es la forma CORRECTA de hacer ventana deslizante:
    1. Acceder al message_store del thread actual
    2. Extraer últimos N mensajes usando slicing
    3. Reproducir en nuevo thread con agent.run()
    """
    
    def __init__(self, umbral=10, ventana=5):
        self.umbral = umbral           # Cuando reducir
        self.ventana = ventana         # A cuántos mensajes
        self.contador_reducciones = 0
    
    async def crear_ventana_deslizante(self, thread_antiguo, agent, entrada_nueva):
        """
        Crea ventana deslizante extrayendo del thread REAL
        
        Args:
            thread_antiguo: El thread con todo el historial
            agent: El agente para reproducir mensajes
            entrada_nueva: La nueva entrada a procesar
        """
        # PASO 1: Acceder al message_store real del thread
        todos_los_mensajes = await thread_antiguo.message_store.list_messages()
        cantidad_actual = len(todos_los_mensajes)
        
        print(f"   📊 Historial actual: {cantidad_actual} mensajes")
        
        # PASO 2: Verificar si necesita reducción
        necesita_reduccion = cantidad_actual > self.umbral
        
        if necesita_reduccion:
            self.contador_reducciones += 1
            print(f"\n   🔄 REDUCCIÓN AUTOMÁTICA #{self.contador_reducciones}")
            print(f"      Mensajes actuales: {cantidad_actual} > Umbral: {self.umbral}")
            print(f"      → Reduciendo a últimos {self.ventana} mensajes...")
            
            # PASO 3: Extraer últimos N mensajes
            ultimos_n = todos_los_mensajes[-self.ventana:]
            
            # PASO 4: Crear nuevo thread y reproducir
            thread_nuevo = agent.get_new_thread()
            mensajes_reproducidos = 0
            
            for msg in ultimos_n:
                # Extraer contenido correctamente
                if hasattr(msg, 'content'):
                    msg_text = msg.content if isinstance(msg.content, str) else str(msg.content)
                elif hasattr(msg, 'text'):
                    msg_text = msg.text
                else:
                    msg_text = str(msg)
                
                # Reproducir en nuevo thread
                await agent.run(msg_text, thread=thread_nuevo)
                mensajes_reproducidos += 1
            
            print(f"      ✓ {mensajes_reproducidos} mensajes reproducidos en nuevo thread")
            
            # Retornar el thread nuevo como el "actual"
            return thread_nuevo
        else:
            # No necesita reducción, mantener el thread actual
            return thread_antiguo

print("✅ ReductorAutomaticoConMessageStore definido (usando message_store REAL)")

✅ ReductorAutomaticoConMessageStore definido (usando message_store REAL)


In [8]:
# Simular 15 entradas de bitácora
entradas = [
    "1. Warp 5 activado hacia Sector 441",
    "2. Nave desconocida detectada en pantallas",
    "3. Escudos deflectores al 100%",
    "4. Data iniciando análisis de sensores",
    "5. Nave identificada: Clase Vulcana",
    "6. Establecida comunicación subespacial",
    "7. Intercambio de saludos diplomáticos",
    "8. Actualización de ruta de navegación",
    "9. Data reporta anomalía subespacial",
    "10. Consejero Troi detecta inquietud emocional",
    "11. ⚠️ Sistema monitoreando umbral",
    "12. Worf refuerza protocolos de seguridad",
    "13. Geordi La Forge optimiza escudos",
    "14. Dra. Crusher prepara enfermería",
    "15. Capitán Picard listo para negociaciones"
]

In [10]:
print("\n" + "="*70)
print("🎬 ESCENA: Bitácora Estelar de la USS Enterprise - VENTANA DESLIZANTE REAL")
print("Situación: Sistema de bitácora acumulando mensajes y aplicando reducciones")
print("Método: Extrayendo DIRECTAMENTE del thread.message_store REAL")
print("="*70)

# Crear reductor que usa message_store real
reductor = ReductorAutomaticoConMessageStore(umbral=10, ventana=5)

# Crear thread para la bitácora
thread_bitacora = agent.get_new_thread()



print(f"\n📌 Configuración:")
print(f"   • Umbral: {reductor.umbral} mensajes")
print(f"   • Ventana: {reductor.ventana} últimos mensajes (cuando reduce)")
print(f"   • Total de entradas: {len(entradas)}\n")

# Procesar todas las entradas
for i, entrada in enumerate(entradas, 1):
    print(f"📝 Entrada: {entrada}")
    
    # Agregar al thread actual
    resultado = await agent.run(entrada, thread=thread_bitacora)
    
    # DEMOSTRACIÓN INTERACTIVA: Mostrar estado cada 5 entradas
    if i % 5 == 0 or i == len(entradas):
        print(f"\n" + "-"*70)
        print(f"📊 CHECKPOINT: Después de {i} entradas")
        print("-"*70)
        
        # Acceder al message_store REAL y contar
        mensajes_actuales = await thread_bitacora.message_store.list_messages() # type: ignore
        print(f"   📋 Total de mensajes en thread: {len(mensajes_actuales)}")
        
        # Mostrar últimos 3 mensajes que hay en el thread
        print(f"\n   🔍 Últimos 3 mensajes en thread:")
        ultimos_3 = mensajes_actuales[-3:]
        for j, msg in enumerate(ultimos_3, 1):
            # Extraer contenido
            if hasattr(msg, 'content'):
                msg_text = msg.content if isinstance(msg.content, str) else str(msg.content) # type: ignore
            elif hasattr(msg, 'text'):
                msg_text = msg.text
            else:
                msg_text = str(msg)
            print(f"      [{j}] {msg_text}...")
        
        # VERIFICAR SI NECESITA REDUCCIÓN
        if len(mensajes_actuales) > reductor.umbral:
            print(f"\n   ⚠️ NECESITA REDUCCIÓN: {len(mensajes_actuales)} > {reductor.umbral}")
            # Aplicar reducción usando message_store real
            thread_bitacora = await reductor.crear_ventana_deslizante(thread_bitacora, agent, entrada)
        else:
            print(f"\n   ✓ Dentro del límite: {len(mensajes_actuales)} <= {reductor.umbral}")
        
        # CONSULTAR AL AGENTE
        print(f"\n   🤖 Consultando estado del thread...")
        test_pregunta = "¿Cuál es el estado actual de la nave? Responde en una línea."
        resultado_checkpoint = await agent.run(test_pregunta, thread=thread_bitacora)
        print(f"\n   💬 Respuesta: {resultado_checkpoint.text}...")
        
        if reductor.contador_reducciones > 0:
            print(f"\n   🔄 Reducciones realizadas hasta ahora: {reductor.contador_reducciones}")

print("\n" + "="*70)
print("📋 RESULTADO FINAL - Después de todas las Entradas")
print("="*70)

# Estado final
mensajes_finales = await thread_bitacora.message_store.list_messages() # type: ignore
print(f"✓ Total de entradas PROCESADAS: {len(entradas)}")
print(f"✓ Mensajes ACTUALES en thread: {len(mensajes_finales)}")
print(f"✓ Mensaje ELIMINADOS: {len(entradas)*2 - len(mensajes_finales)}")
print(f"✓ Reducciones automáticas ejecutadas: {reductor.contador_reducciones}")

# CONSULTA FINAL
print("\n" + "="*70)
print("🎯 CONSULTA FINAL - Estado Después de Ventana Deslizante")
print("="*70)

consulta_final = "Has procesado 15 eventos de bitácora, pero solo recuerdas los últimos (después de ventana deslizante). ¿Cuál es el RESUMEN de la situación de la nave?"
resultado_final = await agent.run(consulta_final, thread=thread_bitacora)

print(f"\n💬 Respuesta del Sistema (con memoria optimizada por ventana deslizante):")
print("="*70)
print(resultado_final.text)
print("="*70)

print(f"\n✨ INSIGHT: El agente procesó {len(entradas)} eventos")
print(f"   pero mantiene {len(mensajes_finales)} en thread activo (ventana deslizante REAL)")
print(f"   Método: Extracción directa de message_store - filosofía Task 6 ✅")


🎬 ESCENA: Bitácora Estelar de la USS Enterprise - VENTANA DESLIZANTE REAL
Situación: Sistema de bitácora acumulando mensajes y aplicando reducciones
Método: Extrayendo DIRECTAMENTE del thread.message_store REAL

📌 Configuración:
   • Umbral: 10 mensajes
   • Ventana: 5 últimos mensajes (cuando reduce)
   • Total de entradas: 15

📝 Entrada: 1. Warp 5 activado hacia Sector 441
📝 Entrada: 2. Nave desconocida detectada en pantallas
📝 Entrada: 3. Escudos deflectores al 100%
📝 Entrada: 4. Data iniciando análisis de sensores
📝 Entrada: 5. Nave identificada: Clase Vulcana

----------------------------------------------------------------------
📊 CHECKPOINT: Después de 5 entradas
----------------------------------------------------------------------
   📋 Total de mensajes en thread: 10

   🔍 Últimos 3 mensajes en thread:
      [1] Registro actualizado: Teniente Comandante Data iniciando análisis de datos de sensores sobre nave desconocida. Resultados preliminares pendientes....
      [2] 5. Nav

## 🔬 Prueba 3: Simulación de Presupuesto de Tokens

### Concepto
En lugar de contar mensajes, contamos **tokens** (unidades de texto). Los mensajes largos usan más tokens.

**Estimación aproximada**: 4 caracteres ≈ 1 token

Vamos a simular un presupuesto de **1000 tokens** y ver cuántos mensajes caben.

### Crear Bitácora con Entradas de Longitud Variable

In [11]:
async def crear_bitacora_variable():
    """Crea una bitácora con entradas de longitud variable"""
    print("📊 Creando bitácora con entradas de longitud variable...\n")
    
    bitacora2 = agent.get_new_thread()
    
    lista_entradas = [
        "Entrada corta 1: Warp 5",
        "Entrada media 2: Detectada nave Klingon en el cuadrante alfa. Escudos al 100%. Sin hostilidades detectadas.",
        "Entrada 3: Cambio de rumbo",
        "Entrada larga 4: El Teniente Comandante Data ha analizado las anomalías subespaciales del sector 001. Los datos indican una posible interferencia temporal causada por una singularidad cuántica. Se recomienda precaución extrema al acercarse a las coordenadas especificadas. El Consejero Troi ha notado perturbaciones emocionales entre la tripulación.",
        "Entrada 5: Todo nominal",
        "Entrada extensa 6: Reunión del alto mando completada. El Capitán Picard ha autorizado la misión diplomática con los Romulanos. La Doctora Crusher ha preparado los protocolos médicos necesarios. Worf ha reforzado la seguridad en todos los niveles.",
        "Entrada breve 7: Factor de curvatura 6",
        "Entrada moderada 8: Negociaciones con la Federación en curso. Situación estable.",
        "Entrada 9: Sensores a largo alcance activados",
        "Entrada larga 10: El Comandante Riker informa que la misión de exploración del planeta clase M ha sido exitosa. Se han identificado nuevas formas de vida vegetales con propiedades bioquímicas únicas. La Dra. Crusher solicita muestras adicionales para análisis en el laboratorio principal. Geordi La Forge ha detectado un campo electromagnético anómalo en la región polar norte.",
        "Entrada 11: Mantenimiento rutinario completado",
        "Entrada extensa 12: Reporte de ingeniería del Teniente La Forge: Los núcleos de curvatura operan al 98% de eficiencia. Se han reemplazado tres convertidores EPS en las cubiertas 7, 12 y 15. El blindaje de antimateria requiere recalibración menor. Todas las modificaciones han sido documentadas en el registro técnico. Estimación de tiempo para mantenimiento preventivo: 6 horas.",
        "Entrada corta 13: Escudos deflectores óptimos",
        "Entrada moderada 14: Primera oficial reporta que la tripulación está lista para la siguiente fase de la misión.",
        "Entrada larga 15: Conferencia con el Almirante de la Flota Estelar completada vía subespacial. Se ha autorizado la extensión de nuestra misión de exploración por 30 días adicionales. Prioridades actualizadas: cartografía estelar del sector 441, investigación de fenómenos de plasma, y establecimiento de relaciones diplomáticas con civilizaciones emergentes. El Consejero Troi ha programado sesiones de bienestar para mantener la moral de la tripulación."
    ]
    
    # Registrar todas las entradas
    contador = 0
    for entrada in lista_entradas:
        result = await agent.run(entrada, thread=bitacora2)
        contador += 1
    
    print(f"✅ Total de entradas registradas: {contador}\n")
    return lista_entradas

# Ejecutar
lista_entradas = await crear_bitacora_variable()

📊 Creando bitácora con entradas de longitud variable...

✅ Total de entradas registradas: 15



### Calcular Tokens Estimados

In [12]:
def estimar_tokens(texto):
    """Estimar cantidad de tokens (aproximado: 4 caracteres = 1 token)"""
    return len(texto) // 4

# Calcular tokens totales
tokens_totales = sum(estimar_tokens(entrada) for entrada in lista_entradas)

print("💾 Análisis de Tokens:")
print("-" * 65)
for i, entrada in enumerate(lista_entradas, 1):
    tokens = estimar_tokens(entrada)
    print(f"   Entrada {i}: {tokens:4d} tokens - {entrada[:50]}...")

print("-" * 65)
print(f"   TOTAL ESTIMADO: {tokens_totales} tokens")

💾 Análisis de Tokens:
-----------------------------------------------------------------
   Entrada 1:    5 tokens - Entrada corta 1: Warp 5...
   Entrada 2:   26 tokens - Entrada media 2: Detectada nave Klingon en el cuad...
   Entrada 3:    6 tokens - Entrada 3: Cambio de rumbo...
   Entrada 4:   87 tokens - Entrada larga 4: El Teniente Comandante Data ha an...
   Entrada 5:    5 tokens - Entrada 5: Todo nominal...
   Entrada 6:   61 tokens - Entrada extensa 6: Reunión del alto mando completa...
   Entrada 7:    9 tokens - Entrada breve 7: Factor de curvatura 6...
   Entrada 8:   20 tokens - Entrada moderada 8: Negociaciones con la Federació...
   Entrada 9:   11 tokens - Entrada 9: Sensores a largo alcance activados...
   Entrada 10:   94 tokens - Entrada larga 10: El Comandante Riker informa que ...
   Entrada 11:   11 tokens - Entrada 11: Mantenimiento rutinario completado...
   Entrada 12:   94 tokens - Entrada extensa 12: Reporte de ingeniería del Teni...
   Entrada 13:   11 toke

### Aplicar Presupuesto de Tokens (400 tokens)

In [13]:
# Calcular cuántas entradas caben en un presupuesto de 450 tokens
presupuesto_tokens = 400
entradas_en_presupuesto = 0
tokens_usados = 0

print(f"\n🎯 Aplicando presupuesto de {presupuesto_tokens} tokens...")
print("-" * 65)

# Contar desde la más reciente
for entrada in reversed(lista_entradas):
    tokens_entrada = estimar_tokens(entrada)
    if tokens_usados + tokens_entrada <= presupuesto_tokens:
        entradas_en_presupuesto += 1
        tokens_usados += tokens_entrada
        print(f"   ✓ Entrada incluida ({tokens_entrada} tokens) - Total acumulado: {tokens_usados}")
    else:
        print(f"   ✗ Entrada excluida ({tokens_entrada} tokens) - Excedería presupuesto")
        break

print("-" * 65)
print(f"\n📊 Resultado del Presupuesto:")
print(f"   - Presupuesto: {presupuesto_tokens} tokens")
print(f"   - Entradas que caben: {entradas_en_presupuesto} de {len(lista_entradas)}")
print(f"   - Tokens utilizados: {tokens_usados}")
print(f"   - Tokens disponibles: {presupuesto_tokens - tokens_usados}")


🎯 Aplicando presupuesto de 400 tokens...
-----------------------------------------------------------------
   ✓ Entrada incluida (113 tokens) - Total acumulado: 113
   ✓ Entrada incluida (27 tokens) - Total acumulado: 140
   ✓ Entrada incluida (11 tokens) - Total acumulado: 151
   ✓ Entrada incluida (94 tokens) - Total acumulado: 245
   ✓ Entrada incluida (11 tokens) - Total acumulado: 256
   ✓ Entrada incluida (94 tokens) - Total acumulado: 350
   ✓ Entrada incluida (11 tokens) - Total acumulado: 361
   ✓ Entrada incluida (20 tokens) - Total acumulado: 381
   ✓ Entrada incluida (9 tokens) - Total acumulado: 390
   ✗ Entrada excluida (61 tokens) - Excedería presupuesto
-----------------------------------------------------------------

📊 Resultado del Presupuesto:
   - Presupuesto: 400 tokens
   - Entradas que caben: 9 de 15
   - Tokens utilizados: 390
   - Tokens disponibles: 10


## 🎯 Aprendizaje Clave y Mejores Prácticas

### 📚 Conceptos Fundamentales

1. **Ventana de Contexto Limitada**
   - Todos los sistemas tienen límites de memoria
   - La USS Enterprise necesita gestionar eficientemente sus bitácoras

2. **Estrategias de Gestión**
   - **Ventana Deslizante**: Cantidad fija de mensajes (ej: últimos 5)
   - **Basado en Tokens**: Tamaño fijo de contexto (ej: 1000 tokens)
   - **Híbrido**: Combinar ambos enfoques

3. **Truncamiento y Reproducción**
   - Crear nuevos threads con contexto esencial
   - Reproducir mensajes recientes para mantener continuidad
   - Usar `all_messages[-N:]` para extraer últimos N mensajes

### 🚀 En Producción

```python
# Ejemplo de ventana deslizante en producción
WINDOW_SIZE = 10

# Extraer mensajes recientes
recent_messages = conversation_history[-WINDOW_SIZE:]

# Crear nuevo thread con contexto reciente
new_thread = agent.get_new_thread()
for msg in recent_messages:
    await agent.run(msg, thread=new_thread)
```

### 🖖 Sabiduría de la Flota Estelar

> *"La eficiencia lógica requiere optimización de recursos. Un sistema que no gestiona su memoria está destinado al fallo."* - Spock

---

## ✅ Completada

Has aprendido a:
- ✅ Gestionar conversaciones largas con ventanas deslizantes
- ✅ Calcular y aplicar presupuestos de tokens
- ✅ Truncar y reproducir contexto en nuevos threads
- ✅ Comparar estrategias de gestión de contexto

🌌 **Próxima misión**: Explora técnicas avanzadas de optimización de contexto.

🖖 **Fin de transmisión - Memoria de bitácora optimizada**